## 5.1 The basic particle filter

입자필터는 순차 모테카를로 방법을 상태공간 모델에 적용하는 방법이며 중요도 샘플링의 연장선으로 생각하면 이해하기 쉽습니다. 따라서 본 장의 시작은 중요도 샘플링에 대해 간략히 복습하는 것으로 시작하겠습니다.

다음과 같은 평균 값을 구하고자 한다고 합시다.

$$E_\pi(f(X))=\int f(x)\pi(x)dx\tag{5.1}$$

만약 $g$는 $g(x)=0$이면 $\pi(x)=0$인 특성을 갖는 중요도 밀도라고 한다면 위의 식은 다음과 같이 쓸 수 있습니다.

$$E_\pi(f(X))=\int f(x)\frac{\pi(x)}{g(x)}g(x)dx=E_g(f(X)w^*(X))$$

여기서 $w^*(x)=\pi(x)/g(x)$는 중요도 함수라고 부릅니다. 위 식은 분포 $g$로부터 $N$개의 랜덤 샘플을 추출하여 원하는 평균 값을 근사할 수 있음을 말해줍니다.

$$\frac{1}{N}\sum_{i=1}^N f(x^{(i)})w^*(x^{(i)})\approx E_\pi(f(X))\tag{5.2}$$

베이지안 방법에서는 목표 밀도에 대해 정규화 상수 까지만 평가할 수 있습니다. 즉, 알지 못하는 상수 $C$에 대하여 $C\cdot\pi(x)$만 계산할 수 있습니다. 이는 불행히도 중요도 함수 또한 동일한 상수인 $C$까지만 평가할 수 있으며, 식 (5.2)를 직접 사용할 수 없음을 의미합니다. 하지만, $f(x)=C$를 만족하는 $\tilde{w}^{(i)}=Cw^*(x^{(i)})$를 정의하면, 식(5.2)는 다음과 같이 표현할 수 있습니다.

$$\frac{1}{N}\sum_{i=1}^N Cw^*(x^{(i)})=\frac{1}{N}\sum_{i=1}^N \tilde{w}^{(i)} \approx E_\pi(C)=C\tag{5.3}$$

$\tilde{w}^{(i)}$를 구할 수 있기 때문에, 식(5.3)를 사용하면 $C$를 추정할 수 있습니다. 또한 식 (5.1)를 평가하는 것이 목적이라면, 아래와 같은 이유로 상수 $C$를 정확히 구하지 않아도 됩니다.

$$E_\pi(f(X))\approx\frac{1}{N}\sum_{i=1}^{N}f(x^{(i)}w^*(x^{(i)})$$

$$=\frac{\frac{1}{N}\sum_{i=1}^{N}f(x^{(i)})\tilde{w}^{(i)}}{C}\approx\frac{\sum_{i=1}^{N}f(x^{(i)})\tilde{w}^{(i)}}{\sum_{i=1}^{N}\tilde{w}^{(i)}}$$

$$=\sum_{i=1}^{N}f(x^{(i)})w^{(i)}$$

여기서 $w^{(i)}=\tilde{w}^{(i)}/\sum_{j=1}^N\tilde{w}^{(j)}$입니다. 여기서 다음의 사항을 알아야 합니다. (1) 가중치 $w^{(i)}$의 합은 1이다, (2) 추정치 $E_\pi(f(X))\approx\sum_{i=1}^N f(x^{(i)})w^{(i)}$는 모든 함수 $f$에 대하여 유효하다. 따라서 가중치 $w^{(1)},\dots,w^{(N)}$과 연관된 샘플들 $x^{(1)},\dots,x^{(N)}$은 목표치 $\pi$에 대한 이산 추정치로 볼 수 있습니다. 다시말해, $\delta_x$를 $x$에서의 단위 질량으로 표현하고, $\hat{\pi}=\sum_{i=1}^Nw^{(i)}\delta_{x^{(i)}}$라고 설정하면 $\pi\approx\hat{\pi}$가 됩니다.

필터링 과정에서 목표 분포는 새로운 관측값을 얻을 때마다 $\pi(\theta_{0:t-1}|y_{1:t-1})$에서 $\pi(\theta_{0:t}|y_{1:t})$로 이동합니다. $\theta_{0:t-1}$들이 $\theta_{0:t}$의 첫 성분임에도 불구하고, $\pi(\theta_{0:t-1}|y_{1:t-1})$는 $\pi(\theta_{0:t}|y_{1:t})$의 주변분포가 아님에 유의해야 합니다. 그러면 $\pi(\theta_{0:t}|y_{1:t})$에 대한 이산 추정치를 얻기 위해, 관측치 $y_t$를 얻을 때마다 어떻게 효율적으로 $\pi(\theta_{0:t-1}|y_{1:t-1})$의 이산 추정치를 업데이트 하느냐가 문제가 됩니다. 모든 시간 $s$에서 $\hat{\pi}_s(\theta_{0:s}|y_{1:s})$를 $\pi(\theta_{0:s}|y_{1:s})$에 대한 추정치라고 합시다. 업데이트 과정은 두 단계로 구성됩니다. (1) $\theta_{0:t}^{(i)}$를 얻기위하여 $\hat{pi}_{t-1}$의 모든 점 $\theta_{0:t-1}^{(i)}$에 대하여 추가 성분 $\theta_t^{(i)}$를 샘플링 합니다. (2) 그에 해당하는 가중치 $w_{t-1}^{(i)}$를 적절한 $w_t^{(i)}$로 업데이트 합니다. 가중치가 부여된 점들 $(\theta_t^{(i)},w_t^{(i)}), i=1,\dots,N$들은 $\hat{\pi}_t$에 대한 새로운 이산 추정치를 제공합니다. 모든 $t$에 대하여 $g_t$는 $\theta_{0:t}$를 생성하기 위해 사용되는 중요도 밀도라고 합시다. 시간 $t$에서는 관측치들 $y_{1:t}$를 가지고 있으므로, $g_t$는 그 관측치들에 따라 변할 것이며, 이러한 의존성을 표현하기 위해 $g_t(\theta_{0:t}|y_{1:t})$라고 쓰겠습니다. $g_t$는 다음과 같은 형태로 표현될 수 있다고 가정하겠습니다. 

$$g_t(\theta_{0:t}|y_{1:t})=g_{t|t-1}(\theta_t|\theta_{0:t-1},y_{1:t})\cdot g_{t-1}(\theta_{0:t-1}|y_{1:t-1})$$

위의 가정을 통해 $\theta_{0:t-1}$를 조합함으로써 $\theta_{0:t}$를 순차적으로 "성장"시킬 수 있습니다. 여기서  $\theta_{0:t-1}$는 시간 $t-1$에서 얻을 수 있는 $g_{t-1}$에서 샘플링 한 것이며, $\theta_t$는 시간 $t$에서 $g_{t|t-1}(\theta_t|\theta_{0:t-1},y_{1:t})$에서 생성한 것입니다. 함수 $g_{t|t-1}$는 중요도 전이 밀도라고 부르겠습니다. $\theta_{0:t}$는 중요도 전이 밀도만 있으면 생성할 수 있습니다. 중요도 밀도를 선택하는 방법은 이 장의 마지막에서 설명하겠습니다. 다음으로 가중치를 어떻게 업데이트 하는지 살펴보겠습니다. 표기를 단순히 하기 위해 상단 첨자를 제거하면 다음과 같습니다. 

$$w_t\propto\frac{\pi(\theta_{0:t}|y_{1:t})}{g_t(\theta_{0:t}|y_{1:t})}\propto\frac{\pi(\theta_{0:t},y_{t}|y_{1:t-1})}{g_t(\theta_{0:t}|y_{1:t})}$$

$$\propto\frac{\pi(\theta_{t},y_{t}|\theta_{0:t-1},y_{1:t-1})\cdot\pi(\theta_{0:t-1}|y_{1:t-1})}{g_{t|t-1}(\theta_{t}|\theta_{0:t-1},y_{1:t})\cdot g_{t-1}(\theta_{0:t-1}|y_{1:t-1})}$$

$$\propto\frac{\pi(y_t|\theta_t)\cdot \pi(\theta_t|\theta_{t-1})}{g_{t|t-1}(\theta_t|\theta_{0:t-1},y_{1:t})}\cdot w_{t-1}$$

따라서, $g_{t|t-1}(\theta_t|\theta_{0:t-1}^{(i)},y_{1:t})$로부터 모든 $i$에 대하여 $\theta_t^{(i)}$를 샘플링한 후, 다음과 같이 정규화 되지 않은 가중치 $\tilde{w}_t^{(i)}$를 계산할 수 있습니다. 

$$\tilde{w}_t^{(i)}=w_{t-1}^{(i)}\cdot \frac{\pi(y_t|\theta_t^{(i)})\cdot \pi(\theta_t^{(i)}|\theta_{t-1}^{(i)})}{g_{t|t-1}(\theta_t^{(i)}|\theta_{0:t-1}^{(i)},y_{1:t}}\tag{5.4}$$

식(5.4)의 좌측의 분수 또는 그에 비례하는 값을 증분 가중치라고 부릅니다. 업데이트 과정에서 마지막 단계는 정규화 되지 않은 가중치를 정규화 하는 것입니다.

$$w_t^{(i)}=\frac{\tilde{w}_t^{(i)}}{\sum_{j=1}^N\tilde{w}_t^{(j)}}$$

업데이트를 많이 수행한 뒤 $\hat{\pi}_t$에 대한 일부 점들이 상대적으로 큰 가중치를 가질 것이며 나머지 점들은 무시할만큼 작은 가중치를 가지게 될 것입니다. 이는 몬테카를로 추정이 약화되도록 만듭니다. 이러한 현상을 제어하기 위하여 다음과 같이 정의되는 유효 샘플 크기를 사용하면 시간에 따른 유용한 감시 기준이 될 수 있습니다.

$$N_{eff}=\left(\sum_{i=1}^N (w_t^{(i)})^2\right)^{-1}$$

상기 유효 샘플 크기는 $N$ (모든 가중치가 같은 크기인 경우)에서 1 (한개의 가중치 크기가 1인 경우)의 범위를 갖습니다. $N_{eff}$가 허용기준 $N_0$에 다다르면, 리샘플 단계를 수행하게 됩니다. 이는 다양한 방법으로 수행 가능합니다. 가장 단순한 방법은 다항 리샘플(multinomial resample)이며, $\hat{\pi}$에서 $N$개의 랜덤 샘플을 추출하고, 균일한 가중치를 갖는 샘플된 점들을 사용하여 목표에 대한 새로운 이산 추정을 수행하는 것입니다. 리샘플 단계는 추정 분포 $\hat{\pi}_t$의 평균 값을 변화시키지 않지만, 몬테 카를로 분산은 증가시킵니다. 분산의 증가를 최소화 하기 위해 다른 리샘플 알고리즘들이 개발되어 왔으며, 이러한 방법들은 분산 증가 관점에서 다항 리샘플보다 효율적입니다. 이러한 방법들 중 가장 널리 사용되는 것이 잔차 리샘플입니다. 이 방법은 결정론 적으로 $ㅑ=1,\dots,N$에 대하여 $[Nw_t^{(i)}]$개의 $\theta_{0:t}^{(i)}$의 복사본을 생성하고, 다항 분포를 갖는 $(R_1,\dots,R_N)$에 대하여 $R_i$개의 $\theta_{0:t}^{(i)}$의 복사본을 생성하는 과정으로 구성됩니다. 크기와 확률 변수들은 $N-M$과 $(\bar{w}^{(1)},\dots,\bar{w}^{(N)})$으로 주어지며, 이에 해당하는 값들은 다음과 같습니다. 

$$M=\sum_{i=1}^N [Nw_t^{(i)}]$$

$$\bar{w}^{(I)}=\frac{Nw_t^{(i)}-[Nw_t^{(i)}]}{N-M},\;\;\;\;t=1,\dots,N$$

알고리즘 5.1은 기본 입자 필터를 요약한 것입니다. 알고리즘의 순차적 특징을 다시한번 강조합니다. "for"루프에서의 변하는 값들은 새로운 관측 데이터 $y_t$에 대하여 $\hat{\pi}_{t-1}$에서 $\hat{\pi}_{t}$로의 업데이트를 의미합니다. 따라서 임의의 시간 $t\leq T$에서 현재 필터링 분포의 추정치 $\hat{\pi}_t$를 구할 수 있습니다.

## 알고리즘 5.1 : 입자 필터 알고리즘 요약

--------------------------------------------------------------------------------------------------------
<div><h4>0. 초기화 : $\pi(\theta_0)$에서 독립적으로 $N$개의 변수 $\theta_0^{(1)},\dots,\theta_0^{(N)}$를 샘플링 하고 다음과 같이 가중치를 설정한다.</h4></div>
<br>
<div>
$$w_0^{(i)}=N^{-1},\;\;\;\;i=1,\dots,N$$
</div>
<br>
<div><h4>1. For $t=1,\dots,T$</h4></div>

<div><h5>&nbsp;&nbsp;1.1) For $i=1,\dots,N$</h5></div>


<div><h6>&nbsp;&nbsp;&nbsp;&nbsp;* 중요도 전이 밀도 $g_{t|t-1}(\theta_t|\theta_{0:t-1}^{(i)},y_{1:t})$에서 변수 $\theta_t^{(i)}$를 샘플링 하고 샘플링한 변수를 사용해 다음과 같이 변수 세트를 업데이트 한다.</h6></div>

<br>
<div>     
$$\theta_{0:t}^{(i)}=(\theta_{0:t-1}^{(i)},\theta_{t}^{(i)})$$
</div>     
<br>     
<div><h6>&nbsp;&nbsp;&nbsp;&nbsp;* 업데이트한 변수 세트와 기존 가중치 및 중요도 전이 밀도를 사용하여 다음과 같이 비정규화 가중치를 추정 한다.</h6></div>

<br>
<div>$$\tilde{w}_t^{(i)}=w_{t-1}^{(i)}\cdot\frac{\pi(\theta_t^{(i)},y_t|\theta_{t-1}^{(i)}}{g_{t|t-1}(\theta_t^{(i)}|\theta_{0:t-1}^{(i)},y_{1:t}}$$</div>
<br>    
    
<div><h5>&nbsp;&nbsp;1.2) 다음과 같이 가중치를 정규화 한다.</h5></div>
  
<br>
<div>$$w_t^{(i)}=\frac{\tilde{w}_t^{(i)}}{\sum_{j=1}^N \tilde{w}_t^{(j)}}$$</div>
<br>  
  
<div><h5>&nbsp;&nbsp;1.3) 다음과 같이 유효샘플크기를 계산한다.</h5></div>
  
<br>  
<div>$$N_{eff}=\left(\sum_{i=1}^N (w_t^{(i)})^2\right)^{-1}$$</div>
<br>  
  
<div><h5>&nbsp;&nbsp;1.4) $N_{eff} < N_0$이면 다음과 같이 리샘플을 수행한다.</h5></div>
  
  
<div><h6>&nbsp;&nbsp;&nbsp;&nbsp;* 이산 분포로 부터 $N$개의 샘플을 추출한다.</h6></div>
    
<br>    
<div>$$P(\theta_{0:t}=\theta_{0:t}^{(i)})=w_t^{(i)},\;\;\;\;i=1,\dots,N$$</div>
<br>    
    
<div><h6>&nbsp;&nbsp;&nbsp;&nbsp;그리고 이 샘플들을 다시 라벨링 한다.</h6></div>
    
<br>    
<div>$$\theta_{0:t}^{(1)},\dots,\theta_{0:t}^{(N)}$$</div>
<br>    
    
<div><h6>&nbsp;&nbsp;&nbsp;&nbsp;* 가중치를 초기화 한다 : $w_t^{(i)}=N^{-1},\;\;i=1,\dots,N$</h6></div>
    
    
<div><h5>&nbsp;&nbsp;1.5) $\hat{\pi}_t=\sum_{i=1}^N w_t^{(i)}\delta_{\theta_{0:t}^{(i)}}$로 설정한다.</h5></div>

----------------------------------------------------------------------------------------------------


시간 $t$에서 필터링 분포 $\pi(\theta_t|y_{0:t})$의 이산 추정치는 $\hat{\pi}_t$의 주변분포를 구하면 얻을 수 있습니다. $\hat{\pi}_t=\sum_{i=1}^N w^{(i)}\delta_{\theta_{0:t}^{(i)}}$이면, 처음 t 성분인 $\theta_{0:t}^{(i)}$는 버리고 $\theta_t^{(i)}$만 남겨 다음을 얻을 수 있습니다.

$$\pi(\theta_t|y_{1:t})\approx\sum_{i=1}^N w^{(i)}\delta_{\theta_t^{(i)}}$$

<table border=